In [1]:
# Import required libraries
import numpy as np  # For numerical operations
import os  # For file and directory operations
import librosa  # For audio processing
from keras.models import Model  # For creating the Keras model
from keras.layers import Input, LSTM, Dense, Dropout, Activation, Bidirectional  # For layers in the model
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam  # For the optimizer
from tensorflow.keras.utils import to_categorical  # For one-hot encoding
from sklearn.model_selection import train_test_split  # For splitting the dataset

In [2]:
# Function to extract MFCC features from an audio file
def mfcc_extract(wavfile_name):
    y, sr = librosa.load(wavfile_name)
    mfcc = np.mean(librosa.feature.mfcc(y=y, sr=sr, n_mfcc=40).T, axis=0)
    return mfcc

In [3]:
# Initialize lists for storing data and labels
emotion_labels = []
gender_labels = []
data = []

# Loop through each file in the dataset directory
for dirname, _, filenames in os.walk('d:/PROJECTS/SIH_HACKATHON/ravdess_dataset/data'):
    for filename in filenames:
        try:
            actor_number = int(filename.split("-")[-1].split(".")[0])
            emotion_labels.append(int(filename[7:8]) - 1)
            gender_labels.append(0 if actor_number % 2 == 0 else 1)
            wavfile_name = os.path.join(dirname, filename)
            data.append(mfcc_extract(wavfile_name))
        except Exception as e:
            print(f"Failed path: {wavfile_name}, Error: {e}")

In [ ]:
0# Convert lists to numpy arrays for use in ML
data_array = np.asarray(data)
emotion_label_array = np.array(emotion_labels)
gender_label_array = np.array(gender_labels)

# One-hot encode the labels
emotion_label_categorical = to_categorical(emotion_label_array)
gender_label_categorical = to_categorical(gender_label_array)

In [ ]:
# Split dataset into training, validation, and test sets
x_data, x_test, y_data_emotion, y_test_emotion, y_data_gender, y_test_gender = train_test_split(
    np.array(data_array), emotion_label_categorical, gender_label_categorical, test_size=0.20, random_state=42)

x_train, x_val, y_train_emotion, y_val_emotion, y_train_gender, y_val_gender = train_test_split(
    x_data, y_data_emotion, y_data_gender, test_size=0.25, random_state=42)  # 0.25 x 0.8 = 0.2

In [ ]:
# Define the LSTM model with two bidirectional LSTM layers
def lstm_model():
    inp = Input(shape=(40, 1))  # Input layer

    # First Bidirectional LSTM layer
    x = Bidirectional(LSTM(128, return_sequences=True))(inp)

    # Second Bidirectional LSTM layer
    x = Bidirectional(LSTM(64, return_sequences=False))(x)

    x = Dense(128)(x)  # Dense layer
    x = Dropout(0.4)(x)  # Dropout layer for regularization
    x = Activation('relu')(x)  # ReLU activation

    x = Dense(128)(x)  # Another dense layer
    x = Dropout(0.4)(x)  # Another dropout layer for regularization
    x = Activation('relu')(x)  # Another ReLU activation

    # Output layers for emotion and gender
    emotion_output = Dense(8, activation='softmax', name='emotion_output')(x)
    gender_output = Dense(2, activation='softmax', name='gender_output')(x)

    # Final model
    model = Model(inputs=inp, outputs=[emotion_output, gender_output])

    # Compile the model
    model.compile(loss={"emotion_output": "categorical_crossentropy", "gender_output": "categorical_crossentropy"},
                  optimizer='Adam',
                  metrics=['accuracy'])
    return model


In [ ]:
# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=90)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.001)

In [ ]:
# Create the model
model_t = lstm_model()

# Train the model
history = model_t.fit(
    np.expand_dims(x_train, -1),
    {"emotion_output": y_train_emotion, "gender_output": y_train_gender},
    validation_data=(np.expand_dims(x_val, -1), {"emotion_output": y_val_emotion, "gender_output": y_val_gender}),
    epochs=250,
    shuffle=True,
    callbacks=[early_stopping, reduce_lr]
)


Epoch 1/250
54/54 [==============================] - 11s 45ms/step - loss: 2.5017 - emotion_output_loss: 2.0587 - gender_output_loss: 0.4430 - emotion_output_accuracy: 0.1714 - gender_output_accuracy: 0.8054 - val_loss: 2.3356 - val_emotion_output_loss: 1.9706 - val_gender_output_loss: 0.3650 - val_emotion_output_accuracy: 0.1875 - val_gender_output_accuracy: 0.8455 - lr: 0.0010
Epoch 2/250
54/54 [==============================] - 1s 16ms/step - loss: 2.3734 - emotion_output_loss: 2.0035 - gender_output_loss: 0.3699 - emotion_output_accuracy: 0.1922 - gender_output_accuracy: 0.8332 - val_loss: 2.2552 - val_emotion_output_loss: 1.9254 - val_gender_output_loss: 0.3298 - val_emotion_output_accuracy: 0.2639 - val_gender_output_accuracy: 0.8472 - lr: 0.0010
Epoch 3/250
54/54 [==============================] - 1s 18ms/step - loss: 2.2578 - emotion_output_loss: 1.9192 - gender_output_loss: 0.3387 - emotion_output_accuracy: 0.2345 - gender_output_accuracy: 0.8570 - val_loss: 2.1706 - val_emoti

In [ ]:
model_t.save("emotion_acc-82.h5")

In [ ]:

# Evaluate on test set
test_metrics = model_t.evaluate(
    np.expand_dims(x_test, -1),
    {"emotion_output": y_test_emotion, "gender_output": y_test_gender}
)
print(f"Test metrics: {test_metrics}")

# Emotion and Gender labels
emotions = {1: 'neutral', 2: 'calm', 3: 'happy', 4: 'sad', 5: 'angry', 6: 'fearful', 7: 'disgust', 8: 'surprised'}
genders = {0: 'female', 1: 'male'}

18/18 [==============================] - 0s 9ms/step - loss: 1.7904 - emotion_output_loss: 1.6144 - gender_output_loss: 0.1760 - emotion_output_accuracy: 0.8125 - gender_output_accuracy: 0.9722
Test metrics: [1.7904330492019653, 1.6144070625305176, 0.1760261207818985, 0.8125, 0.9722222089767456]


In [ ]:
import tensorflow as tf
load_model = tf.keras.load_model("/content/drive/MyDrive/sih_accuracy/emotion_acc-82.h5")

AttributeError: ignored

In [ ]:
def predict(wavfile_testname):
    test_point = mfcc_extract(wavfile_testname)
    test_point = np.reshape(test_point, newshape=(1, 40, 1))
    predictions = model_t.predict(test_point)

    print("Emotion:", emotions[np.argmax(predictions[0]) + 1])
    print("Gender:", genders[np.argmax(predictions[1])])

In [ ]:
# Emotion labels
